<a href="https://colab.research.google.com/github/5248p/breed/blob/main/MMAction2_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MMAction2 설치

In [ ]:
# cuda, gcc 버전 확인
!nvcc -V

!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==latest+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 26kB/s 
     |████████████████████████████████| 6.6MB 18.2MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 23.6MB 139kB/s 
     |████████████████████████████████| 194kB 16.3MB/s 
Cloning into 'mmaction2'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9306 (delta 0), reused 0 (delta 0), pack-reused 9301
Receiving objects: 100% (9306/9306), 35.30 MiB | 41.94 MiB/s, done.
Resolving deltas: 100% (6600/6600), done.
/content/mmaction

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
0.11.0
10.1
GCC 7.3


## Inference

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

mkdir: cannot create directory ‘checkpoints’: File exists
--2021-02-23 04:38:00--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = '/content/mmaction2/configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

Use load_from_local loader


In [ ]:
# Use the recognizer to do inference
video = '/content/mmaction2/demo/demo.mp4'
label = '/content/mmaction2/demo/label_map_k400.txt'
results = inference_recognizer(model, video, label)

In [ ]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

arm wrestling:  29.616438
rock scissors paper:  10.754841
shaking hands:  9.908401
clapping:  9.189913
massaging feet:  8.305306


## 커스텀 데이터

먼저 Kinetics-400에서 얻은 데이터 세트를 다운로드 해 보겠습니다. 레이블이있는 동영상 30 개를 훈련 데이터 세트로 선택하고 라벨이있는 동영상 10 개를 테스트 데이터 세트로 선택합니다.

In [ ]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2021-02-23 04:54:04--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  7.62MB/s    in 2.3s    

2021-02-23 04:54:08 (7.62 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [ ]:
# Check the directory structure of the tiny data

# Install tree first
!apt-get -q install tree
!tree kinetics400_tiny

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 10 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (149 kB/s)
Selecting previously unselected package tree.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50Zw.mp4
│   ├── D32_1gwq35E.mp4
│   ├── D92m0HsHjcQ.mp4
│   ├── DbX8mPslRXg.mp4
│   ├── FMlSTTpN3VY.mp4
│   ├── h10B9SVE-nk.mp4
│   ├── h

In [ ]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


[`VideoDataset`](./datasets/video_dataset.py)에 정의 된 형식에 따라 각 줄은 공백으로 분할 된 파일 경로와 레이블이있는 샘플 동영상을 나타냅니다.

### config 수정

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmaction2/demo/configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 10

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01))
train_cfg = None
test_cfg = dict(average_clips=None)
optimizer = dict(type='SGD', lr=7.8125e-05, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 30
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = 'kine

### 새로운 recognizer 훈련

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


2021-02-23 04:57:28,612 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
2021-02-23 04:57:28,663 - mmaction - INFO - load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
2021-02-23 04:57:28,665 - mmaction - INFO - Use load_from_local loader
2021-02-23 04:57:28,757 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
2021-02-23 04:57:28,761 - mmaction - INFO - Start running, host: root@d2ed66d984e6, work_dir: /content/tutorial_exps
2021-02-23 04:57:28,762 - mmaction - INFO - workflow: [('train', 1)], max: 30 epochs
/content/mmaction2/mmaction/core/evaluation/eval_ho

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 5.9 task/s, elapsed: 2s, ETA:     0s

2021-02-23 04:58:02,430 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-02-23 04:58:02,431 - mmaction - INFO - 
top1_acc	0.7000
top5_acc	1.0000
2021-02-23 04:58:02,431 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-02-23 04:58:02,437 - mmaction - INFO - 
mean_acc	0.7000
2021-02-23 04:58:02,738 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2021-02-23 04:58:02,740 - mmaction - INFO - Best top1_acc is 0.7000 at 5 epoch.
2021-02-23 04:58:02,741 - mmaction - INFO - Epoch(val) [5][15]	top1_acc: 0.7000, top5_acc: 1.0000, mean_class_accuracy: 0.7000
2021-02-23 04:58:07,372 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:02:46, time: 0.922, data_time: 0.710, memory: 2918, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls: 0.6695, loss: 0.6695, grad_norm: 11.0187
2021-02-23 04:58:08,397 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:02:39, time: 0.205, data_time: 0.003, memory: 2918, top1_acc: 0.6000, top5_acc: 1.0000, loss_cl

...

2021-02-23 04:59:46,112 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-02-23 04:59:46,114 - mmaction - INFO - 
top1_acc	0.9000
top5_acc	1.0000
2021-02-23 04:59:46,118 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-02-23 04:59:46,121 - mmaction - INFO - 
mean_acc	0.9000
2021-02-23 04:59:46,123 - mmaction - INFO - Epoch(val) [20][15]	top1_acc: 0.9000, top5_acc: 1.0000, mean_class_accuracy: 0.9000
2021-02-23 04:59:50,193 - mmaction - INFO - Epoch [21][5/15]	lr: 7.813e-05, eta: 0:01:02, time: 0.813, data_time: 0.598, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.4516, loss: 0.4516, grad_norm: 8.3861
2021-02-23 04:59:51,491 - mmaction - INFO - Epoch [21][10/15]	lr: 7.813e-05, eta: 0:00:59, time: 0.260, data_time: 0.044, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.4542, loss: 0.4542, grad_norm: 9.1114
2021-02-23 04:59:52,409 - mmaction - INFO - Epoch [21][15/15]	lr: 7.813e-05, eta: 0:00:57, time: 0.184, data_time: 0.003, memory: 2918, to

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 5.9 task/s, elapsed: 2s, ETA:     0s

2021-02-23 05:00:19,869 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-02-23 05:00:19,871 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2021-02-23 05:00:19,871 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-02-23 05:00:19,875 - mmaction - INFO - 
mean_acc	1.0000
2021-02-23 05:00:20,177 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_25.pth.
2021-02-23 05:00:20,179 - mmaction - INFO - Best top1_acc is 1.0000 at 25 epoch.
2021-02-23 05:00:20,181 - mmaction - INFO - Epoch(val) [25][15]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000
2021-02-23 05:00:24,374 - mmaction - INFO - Epoch [26][5/15]	lr: 7.813e-05, eta: 0:00:30, time: 0.835, data_time: 0.618, memory: 2918, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.3759, loss: 0.3759, grad_norm: 7.9653
2021-02-23 05:00:25,758 - mmaction - INFO - Epoch [26][10/15]	lr: 7.813e-05, eta: 0:00:27, time: 0.278, data_time: 0.082, memory: 2918, top1_acc: 0.5000, top5_acc: 1.0000, los

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 5.7 task/s, elapsed: 2s, ETA:     0s

2021-02-23 05:00:54,042 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-02-23 05:00:54,043 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2021-02-23 05:00:54,044 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-02-23 05:00:54,045 - mmaction - INFO - 
mean_acc	1.0000
2021-02-23 05:00:54,046 - mmaction - INFO - Epoch(val) [30][15]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000


로그에서 우리는 훈련 과정을 기본적으로 이해하고 recognizer가 얼마나 잘 훈련되었는지 알 수 있습니다.

첫째, ImageNet에서 사전 훈련 된 ResNet-50 backbone이 로드됩니다. 이것은 처음부터 훈련하는 것이 더 많은 비용이 들기 때문에 이미 훈련되어 있는 것을 가져다 사용합니다. 로그는 fc.bias 및 fc.weight를 제외한 ResNet-50 backbone의 모든 weight가 로드되었음을 보여줍니다.

둘째, 우리가 사용하는 데이터 세트가 작기 때문에 TSN 모델을 동작 인식을 위해 finetune했습니다. 원래 TSN은 400 개의 클래스가 포함 된 원래 Kinetics-400 데이터 세트에서 학습되지만 Kinetics-400 Tiny 데이터 세트에는 2 개의 클래스 만 있습니다. 따라서 분류를 위해 사전 훈련 된 TSN의 마지막 FC layer는 weight 모양이 다르며 사용되지 않습니다.

셋째, 훈련 후 recognizer는 기본 평가로 평가됩니다. 결과는 recognizer가 val 데이터 세트에서 100 % top1 정확도와 100 % top5 정확도를 달성 함을 보여줍니다.

## 예측 결과 확인

In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 2.2 task/s, elapsed: 5s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	1.0000
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	1.0000
top1_acc: 1.0000
top5_acc: 1.0000
mean_class_accuracy: 1.0000
